In [1]:
import torch
from torch.utils.data import Dataset
import os
from PIL import Image
import pandas as pd
from torch.utils.data import DataLoader
import numpy as np
import ast
from torchvision.transforms import PILToTensor

In [2]:
class customDataset(Dataset):
    def __init__(self) -> None:
        super().__init__()
        self.dir = "dataset_resized/"
        self.annotations = pd.read_csv(os.path.join(self.dir, "annotations.csv"))

        self.X = self.annotations.filename
        self.parking_lots = self.annotations.parking_lots
        
    # def resize_image(self, img_arr, bboxes, h, w):
    #     """
    #     :param img_arr: original image as a numpy array
    #     :param bboxes: bboxes as numpy array where each row is 'x_min', 'y_min', 'x_max', 'y_max', "class_id"
    #     :param h: resized height dimension of image
    #     :param w: resized weight dimension of image
    #     :return: dictionary containing {image:transformed, bboxes:['x_min', 'y_min', 'x_max', 'y_max', "class_id"]}
    #     """
    #     # create resize transform pipeline
    #     # transform = albumentations.Compose(
    #     #     [albumentations.Resize(height=h, width=w, always_apply=True)],
    #     #     bbox_params=albumentations.BboxParams(format='pascal_voc'))

    #     # transformed = transform(image=img_arr, bboxes=bboxes)
    #     # print("img_arr", img_arr.shape)
    #     # return transformed
    #     height, width, _ = img_arr.shape
    #     moved = np.moveaxis(img_arr, -1, 0)
    #     # print("moved", moved.shape)
    #     img_arr = torch.from_numpy(moved)
    #     transform = Resize(size=(h, w))

    #     # print(h, w, height, width)
    #     image = transform(img_arr)
    #     # print("inside resize", bboxes)
    #     # print("transformed", image.shape)
    #     try:
    #         bboxes[:, 0] = bboxes[:, 0] * (w/width)
    #         bboxes[:, 1] = bboxes[:, 1] * (h/height)
    #         bboxes[:, 2] = bboxes[:, 2] * (w/width)
    #         bboxes[:, 3] = bboxes[:, 3] * (h/height)

    #     except Exception as e:
    #         bboxes[0] = bboxes[0] * (w/width)
    #         bboxes[1] = bboxes[1] * (h/height)
    #         bboxes[2] = bboxes[2] * (w/width)
    #         bboxes[3] = bboxes[3] * (h/height)
    #     transform = ToTensor()
    #     bboxes = transform(bboxes)
    #     return {"image": image, "bboxes": bboxes}

    def __len__(self):
        nsamples = self.X.shape[0]
        return nsamples

    def __getitem__(self, idx):
            if torch.is_tensor(idx):
                idx = idx.tolist()

            self.img_name = os.path.join(self.dir,
                                    self.X[idx])
            self.image = Image.open(self.img_name)
            labeled_bbox = self.parking_lots[idx]
            labeled_bbox = ast.literal_eval(labeled_bbox)
            # print(labeled_bbox)
            bboxes = [x[0:-1] for x in labeled_bbox]
            labels = [x[-1] for x in labeled_bbox]
            transform = PILToTensor()
            self.image = transform(self.image)
            self.bboxes = torch.tensor(labeled_bbox)
            # print(self.bboxes.dtype)
            # print(self.bboxes[:, 0])
            sample = {'image': self.image, 'bounding_boxes': np.asarray(bboxes), 'labels': labels}
            # sample = self.resize_image(img_arr=self.image, bboxes= self.bboxes, h=224, w=224)

            return sample

In [3]:
parkingDataset = customDataset()

In [4]:
dataloader = DataLoader(parkingDataset, batch_size=8, shuffle=True, collate_fn=lambda x: x )

In [5]:
for batch in dataloader:
    sample = batch
print(sample)

[{'image': tensor([[[208, 198, 168,  ..., 162, 165, 164],
         [151, 163, 192,  ..., 159, 162, 164],
         [110, 145, 203,  ..., 158, 161, 167],
         ...,
         [ 33,  33,  33,  ..., 149, 152, 153],
         [ 34,  34,  33,  ..., 145, 141, 152],
         [ 35,  34,  33,  ..., 154, 147, 165]],

        [[204, 199, 178,  ..., 200, 203, 202],
         [150, 167, 202,  ..., 197, 200, 202],
         [111, 150, 213,  ..., 196, 199, 205],
         ...,
         [ 33,  33,  33,  ..., 133, 136, 137],
         [ 34,  34,  33,  ..., 129, 125, 136],
         [ 35,  34,  33,  ..., 138, 131, 149]],

        [[237, 229, 203,  ..., 245, 248, 247],
         [182, 196, 227,  ..., 242, 245, 247],
         [142, 179, 238,  ..., 241, 244, 250],
         ...,
         [ 33,  33,  33,  ..., 100, 103, 104],
         [ 34,  34,  33,  ...,  96,  92, 103],
         [ 35,  34,  33,  ..., 105,  98, 116]]], dtype=torch.uint8), 'bounding_boxes': array([[  0.,  57., 148., 216.],
       [148.,  86., 224.

In [9]:
from torchvision.utils import draw_bounding_boxes
from torchvision.io import read_image
import torchvision

bboxes = sample[0]["bounding_boxes"]
bboxes = torch.tensor(bboxes, dtype=torch.uint8)

print(bboxes.shape)
# bbox = bboxes[:, 0:-1]
# labels = bboxes[:, -1]
# labels = [str(l) for l in labels]
# print(bbox)
labels = sample[0]["labels"]
labels = [str(x) for x in labels]
# bbox = bbox.unsqueeze(0)
print(bboxes)

img = sample[0]["image"]
# img = img.permute(2, 0, 1)
# img = torch.transpose(img, 2, 0, 1)
print(labels)
print(img.shape)
# img = torch.tensor(img, dtype=torch.uint8)
img = draw_bounding_boxes(img, bboxes,
                        colors="green",
                        labels=labels)

# transform this image to PIL image 
img = torchvision.transforms.ToPILImage()(img) 

# display output 
img.show() 

torch.Size([5, 4])
tensor([[  0,  57, 148, 216],
        [148,  86, 224, 188],
        [167,  55, 209,  78],
        [143,  56, 174,  79],
        [106,  52, 146,  83]], dtype=torch.uint8)
['1', '0', '1', '1', '1']
torch.Size([3, 224, 224])


In [9]:
sample[3]

IndexError: list index out of range

In [74]:
sample[0]['bounding_boxes']

array([[ 97.,   0., 175.,  41.],
       [ 95.,  55., 157.,  92.],
       [ 93., 113., 152., 141.],
       [ 92., 146., 170., 167.]])

In [75]:
len(sample)

2

In [76]:
labels

[1, 1, 1, 1]